## Define Tetris Game

In [ ]:
import numpy as np
import random

class Tetris:
    
    UNDEFINED = -1
    
    TILES = [
        [
            [[0, 2]],  # Tile 0, orientation 0.
            [[0, 1], [0, 1]],  # Tile 0, orientation 1.
        ],
        [
            [[0, 1], [1, 2]],  # Tile 1, orientation 0.
            [[1, 2], [0, 1]],  # Tile 1, orientation 0.
        ],
        [
            [[0, 2], [1, 2]],  # Tile 2, orientation 0.
            [[0, 2], [0, 1]],  # Tile 2, orientation 1.
            [[0, 1], [0, 2]],  # Tile 2, orientation 2.
            [[1, 2], [0, 2]],  # Tile 2, orientation 3.
        ],
        [
            [[0, 2], [0, 2]],  # Tile 4, orientation 0.
        ],
    ]

    def __init__(self, rows, cols, max_tiles, random_seed):
        self.rows, self.cols = rows, cols
        self.max_tiles = max_tiles
        self.random_seed = random_seed
        
        self.restart()
        
    def restart(self):
        self.gameover = False
        self.tile_count = 0
        self.reward = 0
        self.board = np.full((self.rows, self.cols), Tetris.UNDEFINED)
        self.current_tile = Tetris.UNDEFINED
        self.tile_x = Tetris.UNDEFINED
        self.tile_y = Tetris.UNDEFINED
        self.tile_orientation = Tetris.UNDEFINED
        
        # Create predefined tile sequence, used if stochastic_prob=0
        rand_state = random.getstate()
        random.seed(self.random_seed)
        self.tile_sequence = [random.randint(0, len(Tetris.TILES) - 1) 
                              for x in range(self.max_tiles)]
        random.setstate(rand_state)
        
        self.next_tile()
        
    def next_tile(self):
        if self.tile_count < self.max_tiles:
            self.current_tile = self.tile_sequence[self.tile_count]
            self.tile_x = self.cols // 2
            self.tile_y = self.rows
            self.tile_orientation = 0
            self.tile_count += 1
        else:
            self.gameover = True
    
    def move_left(self):
        if self.tile_x - 1 >= 0:
            self.tile_x -= 1
            return True
        else:
            return False
    
    def move_right(self):
        tile_width = len(Tetris.TILES[self.current_tile][self.tile_orientation])
        if self.tile_x + 1 <= self.cols - tile_width:
            self.tile_x += 1
            return True
        else:
            return False
    
    def rotate(self):
        new_orientation = ((self.tile_orientation + 1) 
                           % len(Tetris.TILES[self.current_tile]))
        tile_width = len(Tetris.TILES[self.current_tile][new_orientation])
        if self.tile_x <= self.cols - tile_width:
            self.tile_orientation = new_orientation
            return True
        else:
            return False
        
    def drop(self):
        tile = Tetris.TILES[self.current_tile][self.tile_orientation]
        
        # Find first location where the piece collides with occupied locations.
        self.tile_y = 0
        for x in range(len(tile)):
            # Find first occupied location in this column            
            cury = -1
            for y in range(self.rows -1, -1, -1):
                if self.board[y, self.tile_x + x] > 0:
                    # Calculate the y position for this column if no other columns are taken into account
                    cury = y + 1 - tile[x][0]
                    break
            # Use the largest y position for all columns of the tile
            if self.tile_y < cury:
                self.tile_y = cury

        if self.tile_y + np.max(tile) > self.rows:
            self.gameover = True
            dreward = -100
        else:
            # Change board entries at the newly placed tile to occupied.
            for x in range(len(tile)):
                self.board[self.tile_y + tile[x][0]:self.tile_y + tile[x][1], 
                           x + self.tile_x] = 1

            # Remove full lines.
            removed_lines = 0
            for y in range(self.rows - 1, -1, -1):
                if np.sum(self.board[y, :]) == self.cols:
                    removed_lines += 1
                    for y1 in range(y, self.rows - 1):
                        self.board[y1, :] = self.board[y1 + 1, :]
                    self.board[self.rows - 1, :] = Tetris.UNDEFINED
            
            dreward = 10 ** (removed_lines - 1) if removed_lines > 0 else 0
            
            # Choose the next tile.
            self.next_tile()
        
        self.reward += dreward
        
        return dreward

## Adaptation to Q-Learning

In [ ]:
class TetrisQL(Tetris):
    
    def __init__(self, rows, cols, max_tiles, random_seed):
        super().__init__(rows, cols, max_tiles, random_seed)
    
    def teleport(self, new_x, new_orientation):
        if 0 <= new_orientation < len(Tetris.TILES[self.current_tile]):  # Valid orientation.
            tile_width = len(Tetris.TILES[self.current_tile][new_orientation])
            if 0 <= new_x <= self.cols - tile_width:
                self.tile_x = new_x
                self.tile_orientation = new_orientation
                return True
        return False

In [ ]:
import pygame

tetris = TetrisQL(rows=5, cols=4, max_tiles=50, random_seed=123456)

# GUI parameters.
TILE_SIZE = 20

BLACK = (0, 0, 0)
GREY = (128, 128, 128)
WHITE = (255, 255, 255)
RED =  (255, 0, 0)

# Initialize the game engine.
pygame.init()
pygame.display.set_caption("TETRIS")
screen = pygame.display.set_mode((200 + tetris.cols * TILE_SIZE, 
                                  200 + tetris.rows * TILE_SIZE))
pygame.key.set_repeat(300, 100)  # Set keyboard delay and interval in ms.
font = pygame.font.SysFont("Calibri", 25, True)

# Loop until the window is closed.
running = True
while running:
    
    # Get user input.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                tetris.restart()
            if not tetris.gameover:
                if event.key == pygame.K_LEFT:
                    tetris.teleport(tetris.tile_x - 1, tetris.tile_orientation)
                elif event.key == pygame.K_RIGHT:
                    tetris.teleport(tetris.tile_x + 1, tetris.tile_orientation)
                elif event.key == pygame.K_UP:
                    new_orientation = ((tetris.tile_orientation + 1) 
                                    % len(Tetris.TILES[tetris.current_tile]))
                    tetris.teleport(tetris.tile_x, new_orientation)
                elif event.key == pygame.K_DOWN:
                    tetris.drop()
    
    # Paint game board.
    if pygame.display.get_active():
        screen.fill(WHITE)

        for i in range(tetris.rows):
            for j in range(tetris.cols):
                pygame.draw.rect(screen, 
                    GREY, 
                    [
                        100 + TILE_SIZE * j, 
                        80 + TILE_SIZE * (tetris.rows - i), 
                        TILE_SIZE, 
                        TILE_SIZE,
                    ], 
                    1
                )
                if tetris.board[i][j] > 0:
                    pygame.draw.rect(
                        screen, 
                        BLACK,
                        [
                            101 + TILE_SIZE * j, 
                            81 + TILE_SIZE * (tetris.rows - i), 
                            TILE_SIZE - 2, 
                            TILE_SIZE - 2,
                        ],
                    )
        
        tile = tetris.TILES[tetris.current_tile][tetris.tile_orientation]
        for x in range(len(tile)):
            for y in range(tile[x][0], tile[x][1]):
                pygame.draw.rect(
                    screen,
                    RED,
                    [
                        101 + TILE_SIZE * (x + tetris.tile_x), 
                        81 + TILE_SIZE * (tetris.rows - (y + tetris.tile_y)), 
                        TILE_SIZE - 2,
                        TILE_SIZE - 2,
                    ]
                )
        
        screen.blit(font.render(f"Reward: {tetris.reward}", 
                                True, BLACK), [0, 0])
        screen.blit(font.render(f"Tile {tetris.tile_count}/{tetris.max_tiles}", 
                                True, BLACK), [0, 30])
        if tetris.gameover:
            screen.blit(font.render("G A M E   O V E R", True, RED), 
                        [40, 100 + tetris.rows * TILE_SIZE])
            screen.blit(font.render("Press ESC to try again", True, RED), 
                        [10, 100 + tetris.rows * TILE_SIZE + 30])

    pygame.display.flip()
            
pygame.quit()